# Visualization of Covid-19 cases by Zipcodes with Python/Bokeh

### Import the required modules

In [1]:
import tabula
import requests
import datetime
import geopandas as gpd
import pandas as pd
import json
from dateutil import parser
from filecmp import cmp
from pathlib import Path
from datetime import datetime
from bokeh.io import reset_output, output_notebook, show

from bokeh.plotting import figure
from bokeh.models import Div, Column, Row


#make bokeh output to notebook
reset_output()
output_notebook()

Loading BokehJS ...

### Download the Covid-19 case data from San Diego County

In [2]:
url = 'https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Summary%20of%20Cases%20by%20Zip%20Code.pdf'
pdf= requests.get(url)
with open(f'covid19_in_sd_{datetime.now().strftime("%d-%b-%Y-%H_%M_%S")}.pdf','wb') as f:
    f.write(pdf.content)

demographics_death_url = 'https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Deaths%20by%20Demographics.pdf'
pdf= requests.get(demographics_death_url)
with open(f'demographics_death_in_sd_{datetime.now().strftime("%d-%b-%Y-%H_%M_%S")}.pdf','wb') as f:
    f.write(pdf.content)
    
case_by_ethnicity_url = 'https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Race%20and%20Ethnicity%20Summary.pdf'
pdf= requests.get(case_by_ethnicity_url)
with open(f'count_by_ethnicity_in_sd_{datetime.now().strftime("%d-%b-%Y-%H_%M_%S")}.pdf','wb') as f:
    f.write(pdf.content)

### Check if the pdf is downloaded

In [3]:
!ls *.pdf

case_by_ethnicity_in_sd_25-Apr-2020-21_30_18.pdf
count_by_ethnicity_in_sd_25-Apr-2020-21_36_51.pdf
count_by_ethnicity_in_sd_27-Apr-2020-14_00_02.pdf
covid19_in_sd_2020-04-18.pdf
covid19_in_sd_2020-04-19.pdf
covid19_in_sd_2020-04-20.pdf
covid19_in_sd_2020-04-21.pdf
covid19_in_sd_21-Apr-2020-19_07_58.pdf
covid19_in_sd_21-Apr-2020-22_14_39.pdf
covid19_in_sd_21-Apr-2020-23_42_43.pdf
covid19_in_sd_22-Apr-2020-00_10_08.pdf
covid19_in_sd_22-Apr-2020-10_32_40.pdf
covid19_in_sd_22-Apr-2020-11_00_51.pdf
covid19_in_sd_23-Apr-2020-13_02_49.pdf
covid19_in_sd_23-Apr-2020-15_51_39.pdf
covid19_in_sd_23-Apr-2020-17_31_24.pdf
covid19_in_sd_24-Apr-2020-21_38_49.pdf
covid19_in_sd_24-Apr-2020-21_47_22.pdf
covid19_in_sd_25-Apr-2020-20_35_16.pdf
covid19_in_sd_25-Apr-2020-21_28_58.pdf
covid19_in_sd_25-Apr-2020-21_30_17.pdf
covid19_in_sd_25-Apr-2020-21_36_51.pdf
covid19_in_sd_27-Apr-2020-14_00_01.pdf
demographics_death_in_sd_25-Apr-2020-21_28_59.pdf
demographics_death_in_sd_25-Apr-2020-21_30_18.pdf
demographic

In [17]:

def get_only_unique_pdf(pdfs):
    # get only unique pdfs
    unique_pdfs = [pdfs[0]]
    for file in pdfs[1:]:
        duplicate = False
        for uf in unique_pdfs:
            duplicate |= cmp(file, uf, shallow=True)
        if not duplicate:
            unique_pdfs.append(file)
    return unique_pdfs
    

curr_dir = Path('.')
pdfs = list(curr_dir.glob('**/*.pdf'))
unique_pdfs = get_only_unique_pdf(pdfs)

print(unique_pdfs)
    

[PosixPath('covid19_in_sd_22-Apr-2020-10_32_40.pdf'), PosixPath('case_by_ethnicity_in_sd_25-Apr-2020-21_30_18.pdf'), PosixPath('covid19_in_sd_2020-04-21.pdf'), PosixPath('covid19_in_sd_25-Apr-2020-21_28_58.pdf'), PosixPath('covid19_in_sd_23-Apr-2020-13_02_49.pdf'), PosixPath('covid19_in_sd_24-Apr-2020-21_47_22.pdf'), PosixPath('demographics_death_in_sd_25-Apr-2020-21_36_51.pdf'), PosixPath('count_by_ethnicity_in_sd_27-Apr-2020-14_00_02.pdf'), PosixPath('covid19_in_sd_27-Apr-2020-14_00_01.pdf'), PosixPath('covid19_in_sd_23-Apr-2020-17_31_24.pdf'), PosixPath('covid19_in_sd_2020-04-19.pdf'), PosixPath('demographics_death_in_sd_27-Apr-2020-14_00_02.pdf')]


### Read the Case by ethnicity

In [11]:
raw_data = tabula.read_pdf(Path('count_by_ethnicity_in_sd_25-Apr-2020-21_36_51.pdf'),stream=False,pages=1, multiple_tables=True)[0].iloc[1:,:]
raw_data.columns = raw_data.iloc[1].values
cases_by_ethnicity = raw_data.iloc[2:,:].reset_index(drop=True)
cases_by_ethnicity['Count'] = pd.to_numeric(cases_by_ethnicity['Count'])
cases_by_ethnicity['Rate per 100,000*'] = pd.to_numeric(cases_by_ethnicity['Rate per 100,000*'])
cases_by_ethnicity = cases_by_ethnicity.rename(columns={'Race/Ethnicity (N=2,329)': 'Counts (%)'})
cases_by_ethnicity['Counts (%)'] = pd.to_numeric(cases_by_ethnicity['Counts (%)'].str.replace('%',''))
cases_by_ethnicity['Count2'] = cases_by_ethnicity['Count']**2

In [12]:
from bokeh.models import CustomJS
from bokeh.models.widgets import Select,TextInput
from bokeh.plotting import Figure


source = cases_by_ethnicity.fillna(0).to_dict(orient='list')

source

{'Race and Ethnicity': ['Hispanic or Latino',
  'White',
  'Black or African American',
  'Asian',
  'Pacific Islander',
  'American Indian',
  'Multiple Race',
  'Race/Ethnicity Unknown'],
 'Count': [1110, 811, 115, 227, 27, 8, 31, 614],
 'Counts (%)': [47.7, 34.8, 4.9, 9.7, 1.2, 0.3, 1.3, 0.0],
 'Rate per 100,000*': [96.5, 53.2, 77.9, 62.3, 183.5, 0.0, 27.6, 0.0],
 'Count2': [1232100, 657721, 13225, 51529, 729, 64, 961, 376996]}

In [14]:
code="""
        var column = cb_obj.value;
         line.glyph.{var}.field = column;
        source.change.emit();
     """
#p = figure(x_range =source['Race and Ethnicity'],plot_height=350,plot_width = 1500)
p = Figure(plot_height=350,plot_width = 1500)
line = p.line(x='Count',y= 'Count2',source =source, color='#A6CEE3')
callbacky = CustomJS(args=dict(source=source, line = line), code=code.format(var="y"))

# Add list boxes for selecting which columns to plot on the x and y axis
yaxis_select = Select(title="Other variables", value="Count2",
                      options=['Count','Count2' ])

yaxis_select.js_on_change('value', callbacky)

layout = Column(yaxis_select, p, width=800)

show(layout)


### Read the case count pdf and clean up the data

In [18]:
def tabula_convert_pdf_to_df(pdf):
    
    #Because pdfs are saved with different dateformat in the name, use different method.
    try:
        pdf_download_date = datetime.strptime("".join(str(pdf).split('_')[3:]).split('.')[0],"%d-%b-%Y-%H%M%S") 
    except ValueError as e:
        pdf_download_date = datetime.strptime("".join(str(pdf).split('_')[3:]).split('.')[0],"%Y-%M-%d") 
    
    print(f'Converting {pdf} to df')
    #Since they keep changing the format of the pdf we need to read pdf case by case.
    if pdf_download_date < datetime.strptime('23-04-2020', "%d-%m-%Y"):
        raw_data = tabula.read_pdf(pdf,stream=False,pages=1)[0]
        #data extraction and munging
        df = raw_data.dropna(how='all').reset_index().drop(columns=['index'])
        text_data = df.columns[0]
        df.columns = df.iloc[0].values
        df = df.drop(axis=0, labels=0).rename(columns={'Count':'CaseCount'})
        title = text_data.split('\r')[-1]    
        
        count_data= pd.DataFrame({
                  'ZipCode' : pd.concat([df.iloc[:,0].astype(str),
                                         df.iloc[:,2].astype(str)]), 

                   'CaseCount': pd.concat([df.iloc[:,1],
                                           df.iloc[:,3]])
                 })
        count_data['TotalCount'] = count_data[count_data['ZipCode']=='Total']['CaseCount'].values[0]
        count_data = count_data[count_data['ZipCode'] != 'Total']
        count_data['RatePer100000'] =  "Unknown"
        
        
    else:
        raw_data = tabula.read_pdf(pdf,stream=True,pages=1, lattice=True)
        text_data = raw_data[0].columns[0]
        title = text_data.split('\r')[-1] 
      
        count_data = pd.concat([raw_data[1], raw_data[2]],axis = 0)
        count_data = count_data.rename(columns={'Zip Code':'ZipCode', 'Count':'CaseCount', 'Rate per 100,000*': "RatePer100000"})
        count_data['TotalCount'] = count_data[count_data['ZipCode']=='San Diego County Total']['CaseCount'].values[0]
        
        count_data = count_data[count_data['ZipCode'] != 'San Diego County Total']
        count_data['RatePer100000'] =  count_data['RatePer100000'].replace('**',"Unknown")

    #parse dates
    dates = []
    for _ in title.split():
        try:
            dates.append(parser.parse(_, fuzzy=True))
        except Exception as e:
            pass
    date = dates[0]

    updated_time = datetime.combine(dates[1],datetime.time(dates[2]))


    count_data.dropna(how='any', inplace=True)
    count_data['ReportedDate'] = date 
    count_data['UpdatedDatetime'] = updated_time
    count_data['CaseCount'] = pd.to_numeric(count_data['CaseCount'].str.replace(',',''))
    count_data['ZipCode'] = count_data['ZipCode'].astype('str')
    count_data['TotalCount'] = pd.to_numeric(count_data['TotalCount'].str.replace(',',''))
    
    
    return count_data
 

In [19]:
all_dates_count_data = []   
for pdf in unique_pdfs:
    count_data = tabula_convert_pdf_to_df(pdf)
    total_count = count_data['TotalCount'].unique()[0]
    updated_date = count_data['UpdatedDatetime'].iloc[0].to_pydatetime()
    reported_date = count_data['ReportedDate'].iloc[0].to_pydatetime()
    title = f'Date through {reported_date:%Y-%m-%d}, Updated on {updated_date:%Y-%m-%d %I:%M %p}'
    print(f'{title}, TotalCount: {total_count}')
    all_dates_count_data.append(count_data)
all_dates_count_df = pd.concat(all_dates_count_data)

all_dates_count_df = all_dates_count_df.sort_values(by='ReportedDate', ascending=False).reset_index(drop=True)
all_dates_count_df

Converting covid19_in_sd_22-Apr-2020-10_32_40.pdf to df
Date through 2020-04-20, Updated on 2020-04-21 08:00 AM, TotalCount: 2434


ValueError: time data 'insd25-Apr-2020-213018' does not match format '%Y-%M-%d'

In [79]:
total_count_with_dates = all_dates_count_df[['TotalCount', 'ReportedDate']].drop_duplicates().reset_index()
p = figure(x_axis_type="datetime")
p.grid.grid_line_alpha=0.3
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Total Cases'

p.line(x='ReportedDate',y= 'TotalCount',source = total_count_with_dates, color='#A6CEE3', line_width=5,)
p.circle(x='ReportedDate', y= 'TotalCount',source = total_count_with_dates, color='#A6CEE3',size=10)
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "16pt"
p.yaxis.major_label_text_font_size = "16pt"
p.outline_line_color = None

show(p)

### Get the geojson file of communities

Later in this notebook, I want to plot the data on a map base on zip code geometry. I used the export geojson from https://data.sandiegocounty.gov/Maps-and-Geographical-Resources/Zip-Codes/vsuf-uefy to get the geojson file. In addition to the zip code geometry, it also has the name of the community it belongs to.

In [25]:
county_gpd = gpd.read_file(f'Sandiego_Zip_codes.geojson')
county_gpd.head()

,community,shape_star,shape_stle,zip,geometry
0,Alpine,4149939944.16,326045.262676,91901,"MULTIPOLYGON (((-116.74539 32.96063, -116.7408..."
1,Bonita,273909416.836,113257.374615,91902,"MULTIPOLYGON (((-116.97172 32.70838, -116.9712..."
2,Boulevard,2735681408.51,241725.552214,91905,"MULTIPOLYGON (((-116.23165 32.75083, -116.2280..."
3,Campo,3066759065.62,287410.325075,91906,"MULTIPOLYGON (((-116.35677 32.70460, -116.3572..."
4,Chula Vista,403437442.009,112587.791814,91910,"MULTIPOLYGON (((-117.06354 32.65011, -117.0634..."


I then merge geojson zip code geometry data with case count data. I do a 'how =right' merge with the zip code as the common key. All the right rows (rows in case count per zip code) will be preserved.

In [26]:
merged =county_gpd.merge(all_dates_count_df, right_on = 'ZipCode', left_on = 'zip',
                         how = 'right').drop(columns=['zip']).rename(columns={'community':'CommunityName'})
merged

,CommunityName,shape_star,shape_stle,geometry,ZipCode,CaseCount,TotalCount,RatePer100000,ReportedDate,UpdatedDatetime
0,Alpine,4149939944.16,326045.262676,"MULTIPOLYGON (((-116.74539 32.96063, -116.7408...",91901,4,"2,943",Unknown,2020-04-24,2020-04-25 08:00:00
1,Alpine,4149939944.16,326045.262676,"MULTIPOLYGON (((-116.74539 32.96063, -116.7408...",91901,4,"2,826",Unknown,2020-04-23,2020-04-24 08:00:00
2,Alpine,4149939944.16,326045.262676,"MULTIPOLYGON (((-116.74539 32.96063, -116.7408...",91901,3,"2,643",Unknown,2020-04-22,2020-04-23 08:00:00
3,Alpine,4149939944.16,326045.262676,"MULTIPOLYGON (((-116.74539 32.96063, -116.7408...",91901,3,"2,491",Unknown,2020-04-21,2020-04-22 08:00:00
4,Alpine,4149939944.16,326045.262676,"MULTIPOLYGON (((-116.74539 32.96063, -116.7408...",91901,3,2434,Unknown,2020-04-20,2020-04-21 08:00:00
...,...,...,...,...,...,...,...,...,...,...
721,NaN,NaN,NaN,None,Unknown***,34,"2,643",Unknown,2020-04-22,2020-04-23 08:00:00
722,NaN,NaN,NaN,None,Unknown***,39,"2,491",Unknown,2020-04-21,2020-04-22 08:00:00
723,NaN,NaN,NaN,None,Unknown,31,2434,Unknown,2020-04-20,2020-04-21 08:00:00
724,NaN,NaN,NaN,None,Unknown,31,2325,Unknown,2020-04-19,2020-04-20 08:00:00


In [27]:
latest = merged[merged['ReportedDate'] == merged['ReportedDate'].max()]


## Plot the case counts as function of zip code and community

In [28]:
# Choose subset of the nongeo data.
# If there is no Community found (for ZipCode = Unknown) filll the Community with 'Unknown'

nogeo_data = latest[['ZipCode', 'CaseCount', 'CommunityName','ReportedDate']].fillna('Unknown')


Split the communities into three parts to plot them seperately.

In [29]:
import numpy as np
split0 = nogeo_data.loc[nogeo_data['CommunityName'] == 'San Diego']

## Split the all other remaining 
split1, split2 = np.array_split(nogeo_data.loc[nogeo_data['CommunityName'] != 'San Diego'], 2)

# make groupby to create bokeh nested x range plots
sandiego = split0.groupby(by=['CommunityName', 'ZipCode'])
part1 = split1.groupby(by=['CommunityName', 'ZipCode'])
part2 = split2.groupby(by=['CommunityName', 'ZipCode'])

In [30]:
def create_plot(df):
    source = df.groups
    
    p = figure(plot_height=250, plot_width=800, x_range=df, toolbar_location=None, 
               tooltips=[("Case Count", "@CaseCount_mean"),
                         ("Community Name, ZipCode", "@CommunityName_ZipCode")
                        ]
              )

    p.vbar(x='CommunityName_ZipCode', top='CaseCount_mean', width=1, source=df,
           line_color="white" )

    p.y_range.start = 0
    p.x_range.range_padding = 0.05
    p.xgrid.grid_line_color = None
    p.yaxis.axis_label = "Case Count"
    p.xaxis.major_label_orientation = 22/28
    p.xaxis.group_label_orientation = 22/28
    p.xaxis.major_label_text_font_size = "8pt"
    p.xaxis.group_text_font_size = "10pt"
    p.title.text_font_size = "16pt"
    p.yaxis.axis_label_text_font_size = "16pt"
    p.xaxis.axis_label_text_font_size = "16pt"
    p.yaxis.axis_label_text_font_size = "16pt"
    p.outline_line_color = None
    p.x_range.group_padding = 1.0
    return p

In [31]:
bar_case_count= Column( create_plot(sandiego), create_plot(part1), create_plot(part2))
# groupby are not sorted by bokeh. seems like a bug
#show(bar_case_count)


# Chloropleth Map of Covid 19 cases according to Zip Code in San Diego

In [32]:

# Convert merged data to json. Because there are some case counts from 
# "unknown" they dont have any geo information. Also have to remove datetime fields
# because they cant be serialized with json

latest_json = json.loads(latest.drop(columns = ['ReportedDate', 'UpdatedDatetime']).dropna().to_json())

# Convert to json to str like object because bokeh needs it in this form.
latest_json_str = json.dumps(latest_json)


In [33]:
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer

def chloropleth_case_count(latest_json_str):
    geosource = GeoJSONDataSource(geojson = latest_json_str)

    #Define a sequential multi-hue color palette.
    palette = brewer['YlOrRd'][8]

    #Reverse color order 
    palette = palette[::-1]
    #max(merged['CaseCount'])
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = 0, 
                                     high = 80 )

    tick_labels = {0:"0", 
                   10:"10",
                   20:"20",
                   30:"30",
                   40:"40", 
                   50:"50",
                   60:"60",
                   70:"70",
                   80:">80"
                  }
    #Create color bar. 
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, 
                         width = 500, height = 20,border_line_color=None,
                         location = (0,0), 
                         orientation = 'horizontal', 
                         major_label_overrides = tick_labels)

    #Add hover tool
    hover = HoverTool(tooltips = [ ('Case Count','@CaseCount'),
                                  ('Zip Code', '@ZipCode'),
                                  ('Community Name', '@CommunityName')]
                     )

    #Create figure object.
    map_figure = figure(
        x_axis_location=None, y_axis_location=None,
               plot_height = 1000 , plot_width = 950, 
               toolbar_location = None,
              tools = [hover])

    map_figure.xgrid.grid_line_color = None
    map_figure.ygrid.grid_line_color = None

    #Add patch renderer to figure. 
    map_figure.patches('xs','ys', source = geosource,
              fill_color = {'field' :'CaseCount', 'transform' : color_mapper},
              line_color = 'black', line_width = 0.25, fill_alpha = 1)

    map_figure.title.text_font_size = '16pt'
    #Specify figure layout.
    map_figure.add_layout(color_bar, 'above')

    total_count = latest['TotalCount'][0]
    reported_date = latest['ReportedDate'][0].to_pydatetime()
    updated_date = latest['UpdatedDatetime'][0].to_pydatetime()
    title = f'Date through {reported_date:%Y-%m-%d}, Updated on {updated_date:%Y-%m-%d %I:%M %p}'
    chloropleth_cases = Column(Div(text = title,  style={'font-size': '200%', 'color': 'blue'}),
                    Div(text = f'Total number of cases: {total_count}',style={'font-size': '200%', 'color': 'red'}), 
                    Div(text = f'Number of Cases',style={'font-size': '150%', 'color': 'blue'}), 
                        map_figure)
    return chloropleth_cases


                    
#show(chloropleth_case_count(latest_json_str))



In [34]:
def chloropleth_cases_per_pop(latest_json_str):
    geosource = GeoJSONDataSource(geojson = latest_json_str)

    #Define a sequential multi-hue color palette.
    palette = brewer['YlOrRd'][8]

    #Reverse color order 
    palette = palette[::-1]
    #max(merged['CaseCount'])
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = 0, 
                                     high = 160 )

    tick_labels = {0:"0", 
                   20:"20",
                   40:"40",
                   60:"60",
                   80:"80", 
                   100:"100",
                   120:"120",
                   140:"140",
                   160:">160"
                  }
    #Create color bar. 
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, 
                         width = 500, height = 20,border_line_color=None,
                         location = (0,0), 
                         orientation = 'horizontal', 
                         major_label_overrides = tick_labels)

    #Add hover tool
    hover = HoverTool(tooltips = [ ('Rate Per 100000','@RatePer100000'),
                                  ('Zip Code', '@ZipCode'),
                                  ('Community Name', '@CommunityName')]
                     )

    #Create figure object.
    map_figure = figure(
        x_axis_location=None, y_axis_location=None,
               plot_height = 1000 , plot_width = 950, 
               toolbar_location = None,
              tools = [hover])

    map_figure.xgrid.grid_line_color = None
    map_figure.ygrid.grid_line_color = None

    #Add patch renderer to figure. 
    map_figure.patches('xs','ys', source = geosource,
              fill_color = {'field' :'RatePer100000', 'transform' : color_mapper},
              line_color = 'black', line_width = 0.25, fill_alpha = 1)

    map_figure.title.text_font_size = '16pt'
    #Specify figure layout.
    map_figure.add_layout(color_bar, 'above')

    chloropleth_cases_per_pop = Column( Div(text = 'Cases per 100,000 people',  style={'font-size': '150%', 'color': 'blue'}),
                                  map_figure)
    return chloropleth_cases_per_pop


                    
#show(chloropleth_cases_per_pop(latest_json_str))



In [35]:
collage = Column(chloropleth_case_count(latest_json_str),
               bar_case_count, 
                chloropleth_cases_per_pop(latest_json_str)
                )
#show(collage)

In [36]:
# Generate standlone html documents with the collage of both plots

from bokeh.resources import CDN
from bokeh.embed import file_html

try:
    html = file_html(collage, CDN, 'Covid19 Cases in San Diego with Bokeh')
except Exception as e:
    print(e)
    
updated_date = latest['UpdatedDatetime'][0].to_pydatetime()
with open(f'Covid19_{updated_date:%Y-%m-%d_%I_%M_%p}.html','w') as f:
    f.write(html)

